# Pocking at Ever Larger Language Models
## An introduction for (digital) humanists



## Introduction 

In [ ]:
## Download data

In [6]:
!wget -O animacy.zip https://bl.iro.bl.uk/downloads/59a8c52f-e0a5-4432-9897-0db8c067627c

--2023-04-26 11:50:52--  https://bl.iro.bl.uk/downloads/59a8c52f-e0a5-4432-9897-0db8c067627c
Resolving bl.iro.bl.uk (bl.iro.bl.uk)... 52.211.187.46, 52.209.26.210, 18.202.144.94
Connecting to bl.iro.bl.uk (bl.iro.bl.uk)|52.211.187.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144694 (141K) [application/zip]
Saving to: ‘animacy.zip’

animacy.zip         100%[===================>] 141.30K  --.-KB/s    in 0.06s   

2023-04-26 11:50:53 (2.46 MB/s) - ‘animacy.zip’ saved [144694/144694]



In [8]:
!unzip animacy.zip

Archive:  animacy.zip
  inflating: LwM-nlp-animacy-annotations-machines19thC.tsv  
  inflating: read-me                 


In [9]:
!ls

LICENSE
LwM-nlp-animacy-annotations-machines19thC.tsv
README.md
SAS-Workshop.ipynb
animacy.zip
read-me


In [10]:
!unzip --help

UnZip 6.00 of 20 April 2009, by Info-ZIP.  Maintained by C. Spieler.  Send
bug reports using http://www.info-zip.org/zip-bug.html; see README for details.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files

  -C  match filenames case-insensitively     -L  make (some) na